In [1]:
from src.dataset import StockDataset

In [2]:
dataset = StockDataset.sequential_ds()
dataset = StockDataset.item_ds()

In [3]:
from torch.utils.data import DataLoader, random_split

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size]
)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)

In [4]:
import torch
import torch.nn as nn

from src.models import MyTransformerEncoderLayer, PositionalEncoding


if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"


class MyTransformerEncoderRegressor(nn.Module):
    def __init__(self, d_model: int, nhead: int, dim_feedforward: int, max_len: int=100) -> None:
        super().__init__()
        self.positional_encoding =  PositionalEncoding(d_model, max_len)
        self.transformer_encoder = MyTransformerEncoderLayer(d_model, nhead, dim_feedforward)
        self.regression_head = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, d_model),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = x[:, -1, :]
        x = self.regression_head(x)
        return x

model = MyTransformerEncoderRegressor(9, 3, 128, 100)
model = model.to(device)

In [5]:
from src.train import train

In [6]:
hist = train(model, 10, device, train_dataloader, val_dataloader)

 10%|██                  | 1/10 [00:08<01:19,  8.87s/it]

Epoch: 1. Training loss: 1.213e+00; Validation loss: 4.299e+00
Saving model at: /Users/l4rralde/Desktop/CIMAT/Semestres/Segundo/DeepLearning/Tareas/Tarea5/models/MyTransformerEncoderRegressor_4.298764367679973


 20%|████                | 2/10 [00:16<01:06,  8.28s/it]

Epoch: 2. Training loss: 4.205e-01; Validation loss: 2.630e+00
Saving model at: /Users/l4rralde/Desktop/CIMAT/Semestres/Segundo/DeepLearning/Tareas/Tarea5/models/MyTransformerEncoderRegressor_2.6302962458723194


 30%|██████              | 3/10 [00:24<00:56,  8.08s/it]

...

Epoch: 8. Training loss: 1.786e-01; Validation loss: 1.456e+00
Saving model at: /Users/l4rralde/Desktop/CIMAT/Semestres/Segundo/DeepLearning/Tareas/Tarea5/models/MyTransformerEncoderRegressor_1.456327770110015


 90%|██████████████████  | 9/10 [01:13<00:08,  8.09s/it]

Epoch: 9. Training loss: 1.664e-01; Validation loss: 1.143e+00
Saving model at: /Users/l4rralde/Desktop/CIMAT/Semestres/Segundo/DeepLearning/Tareas/Tarea5/models/MyTransformerEncoderRegressor_1.1426145327942712


100%|███████████████████| 10/10 [01:21<00:00,  8.12s/it]

Epoch: 10. Training loss: 1.608e-01; Validation loss: 2.363e+00
